[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/caer200/ocelotml_coupling/blob/main/eval.ipynb)

## Helper functions to load model and generate the input

In [6]:
from dig.threedgraph.method import SphereNet
from dig.threedgraph.evaluation import ThreeDEvaluator
import torch
import numpy as np
from torch_geometric.data import Data
from pymatgen.core.structure import Structure, Molecule
from torch_geometric.loader import DataLoader

def predict(batch, checkpoint):
    model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
                      hidden_channels=128, out_channels=1, int_emb_size=64,
                      basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
                      num_spherical=3, num_radial=6, envelope_exponent=5,
                      num_before_skip=1, num_after_skip=2, num_output_layers=3)
    model.load_state_dict(torch.load(checkpoint,map_location=torch.device('cpu'), weights_only = False)["model_state_dict"])
    model.eval()
    results = []
    for data in batch:
        prediction = model(data)
        results.append(round(prediction.detach().numpy()[-1][0],3))
    return results

def predict_from_molecule(molecule, checkpoint):
    """
    args: 

    molecule (Molecule): pymatgen Molecule object
    checkpoint (str): path to checkpoint file 

    Description:

    Makes prediction from Molecule object
    """
    R_i = torch.tensor(molecule.cart_coords, dtype=torch.float32)
    z_i = torch.tensor(np.array(molecule.atomic_numbers), dtype=torch.int64)
    data = Data(pos=R_i, z=z_i,)
    batch = DataLoader([data], batch_size=1)
    return predict(batch,checkpoint=checkpoint)

def predict_from_file(filename, checkpoint):
    """
    args: 
    
    filename (str): path to xyz file
    checkpoint (str): path to checkpoint file

    Description:

    Makes prediction from XYZ file with dimer coordinates
    """
    molecule = Molecule.from_file(filename)
    return predict_from_molecule(molecule, checkpoint)

# get the pretrain model 
# ! git clone https://github.com/caer200/ocelotml_coupling.git

### Making HOMO-HOMO coupling prediction from XYZ file. Output in eV.

In [7]:
predict_from_file("./test.xyz","./models/hh.pt")[0]

/Users/k2584788/.local/share/mamba/envs/ocelotml/lib/python3.10/site-packages/dig/threedgraph/utils/geometric_computing.py:47: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1744233393360/work/aten/src/ATen/native/Cross.cpp:66.)
  b = torch.cross(pos_ji, pos_jk).norm(dim=-1) # sin_angle * |pos_ji| * |pos_jk|


0.208

In [1]:
import ocelotml

ModuleNotFoundError: No module named 'ocelotml'